In [ ]:
import json
from functools import partial

import bitsandbytes as bnb
import torch
from datasets import load_dataset
from peft import LoraConfig, get_peft_model
from transformers import (
    AutoProcessor,
    BitsAndBytesConfig,
    PaliGemmaForConditionalGeneration,
    Trainer,
    TrainingArguments,
)
import datetime

run_name = "pali-gemma-448"

# Get the current timestamp
timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")

# Add the timestamp to the run name
run_name_with_timestamp = f"{run_name}_{timestamp}"

model_id = "google/paligemma-3b-pt-448"
processor = AutoProcessor.from_pretrained(model_id)

def collate_fn(processor: AutoProcessor, examples):
    texts = ["Process " for _ in examples]
    labels = [
        json.dumps({k: v for k, v in example.items() if k != "image"})
        for example in examples
    ]
    images = [example["image"].convert("RGB") for example in examples]
    tokens = processor(
        text=texts,
        images=images,
        suffix=labels,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=128,
        tokenize_newline_separately=False,
    )
    return tokens


collate = partial(collate_fn, processor)

dataset = load_dataset("arnaudstiegler/synthetic_us_passports_easy")

In [ ]:
processor.max_length = 128


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_storage=torch.bfloat16,
)

lora_config = LoraConfig(
    r=8,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
    ],
    task_type="CAUSAL_LM",
)
model = PaliGemmaForConditionalGeneration.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map={'':torch.cuda.current_device()}
)
model.gradient_checkpointing_enable()

In [ ]:
model.enable_input_require_grads()
model = get_peft_model(model, lora_config)

model.print_trainable_parameters()
optimizer = bnb.optim.Adam8bit(model.parameters(), lr=1e-4)

In [ ]:
args = TrainingArguments(
    output_dir="/home/ubuntu/out/",
    num_train_epochs=4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=6,
    gradient_accumulation_steps=1,
    warmup_steps=1000,
    learning_rate=1e-4,
    weight_decay=1e-6,
    adam_beta2=0.999,
    logging_steps=50,
    eval_steps=1, # not working anyway
    optim="paged_adamw_8bit",
    save_strategy="steps",
    save_steps=500,
    push_to_hub=True,
    save_total_limit=1,
    bf16=True,
    run_name=run_name_with_timestamp,
    report_to=["wandb"],
    dataloader_pin_memory=False,
    remove_unused_columns=False,
)


In [ ]:
trainer = Trainer(
    model=model,
    train_dataset=dataset['train'],
    eval_dataset=dataset["test"],
    data_collator=collate,
    args=args,
)

In [ ]:
trainer.train()